In [1]:
import pandas as pd
import numpy as np
from numpy.fft import fft
import sys
import os
import logging
sys.path.insert(0, '../')
import features_engineering_functions  as fef
sys.path.insert(0, '../..')
from config import *

# Models parameters, features and constants

In [2]:


'''
files and directory gestion, the test file is csv_test
'''
DIR_CSV_MODEL = DIR_TAGGED_EVT
DIR_SAVE_FEATURES = 'saved_features/'
list_evt_files = os.listdir(DIR_CSV_MODEL)[1:] # le 1 c'est pour éviter le gitkeep

'''
Model hyperparmeters and constants
'''
DUREE_EPOCH = 5
WINDOW_OVERLAP = DUREE_EPOCH / 2
ENTROPY_WINDOWS = [[0,DUREE_EPOCH / 2],[DUREE_EPOCH / 2 , DUREE_EPOCH]]

list_evt_infos_columns = ['evt_name','epoch_nb','label']
list_members = LIST_MODEL_MEMBERS


In [3]:
'''
Time features
'''
list_time_features = ['mean','std','min','max','mean_absolute','up_down']
dic_tf_functions = fef.get_dic_time_features_functions()
list_time_features = fef.get_times_features_name(list_members,list_time_features)

'''
Entropy features
'''
entropy_features = ['ent_tot','ent_left','ent_right']
list_entropy_features = fef.get_entropy_features_name(list_members,entropy_features)

'''
Fourier features and parameters
'''

fourier_window_size = DUREE_EPOCH

list_intervalle_frequence_fourier = [0,5,9,23]
list_fourier_features = fef.get_fourier_features_names(list_intervalle_frequence_fourier,list_members)

'''
wavelet features
'''

'''
All features with meta data
'''
list_all_features = fef.make_all_features_list(list_time_features,list_fourier_features,list_entropy_features,True)
list_all_features = list_all_features + ['correlation', 'spectre_MSG','spectre_MSD']



In [7]:
df_model_all_seizure = pd.DataFrame(columns=list_all_features)
df_model_all_seizure = df_model_all_seizure.astype({"spectre_MSG" : object})
df_model_all_seizure = df_model_all_seizure.astype({"spectre_MSD" : object})
csv_saving_name = 'saved_features_E{}'.format(DUREE_EPOCH)
save_files = False
i,it = 0,0
borne_acc = 0.04
list_file_temp = list_evt_files
print('le nb de fichier à traiter est de {}'.format(len(list_file_temp)))
for i,csv_name in enumerate(list_file_temp):
    print('fichier no ', i)
    df_data_csv,mean_msg,mean_msd = fef.make_model_df_from_csv(csv_name,DIR_CSV_MODEL)

    '''
    We select only files with acceleration mean betweene 1 - ba & 1 + ba
    '''
    if ((1-borne_acc) < mean_msg < (1+borne_acc)) & ((1-borne_acc) < mean_msd < (1+borne_acc)):
        it += 1
        df_data_csv['MSG'] = df_data_csv['MSG'] - df_data_csv['MSG'].mean()
        df_data_csv['MSD'] = df_data_csv['MSD'] - df_data_csv['MSD'].mean()
        df_data_csv['seizure'] = [k > 0 for k in df_data_csv['seizure'].tolist()]

        df_model_one_seizure,nb_epoch = fef.make_model_df(df_data_csv,list_all_features,DUREE_EPOCH)
        df_model_one_seizure = df_model_one_seizure.astype({"spectre_MSG" : object})
        df_model_one_seizure = df_model_one_seizure.astype({"spectre_MSD" : object})

        for epoch in range(nb_epoch):
            df_epoch = fef.get_epoch_df(df_data_csv,epoch,duree_epoch = DUREE_EPOCH)
            df_model_one_seizure.loc[epoch,'epoch_nb'] = epoch
            df_model_one_seizure.loc[epoch,'label'] = df_epoch.seizure.mean()
            df_model_one_seizure.loc[epoch,list_time_features] = fef.get_times_features_value(df_epoch,list_members,dic_tf_functions)
            ff_MSG,spectre_MSG = fef.get_epoch_fourier_features(df_epoch,'MSG',list_intervalle_frequence_fourier)
            ff_MSD,spectre_MSD = fef.get_epoch_fourier_features(df_epoch,'MSD',list_intervalle_frequence_fourier)
            df_model_one_seizure.loc[epoch,list_fourier_features] = ff_MSG + ff_MSD
            df_model_one_seizure.at[epoch,'spectre_MSG'] = spectre_MSG
            df_model_one_seizure.at[epoch,'spectre_MSD'] = spectre_MSD
            correlation = df_epoch[['MSG','MSD']].corr().iloc[0,1]
            ent_MSG = fef.get_entropy_from_epoch(df_epoch['MSG'],300,'MSG')
            ent_MSD = fef.get_entropy_from_epoch(df_epoch['MSD'],300,'MSD')
            df_model_one_seizure.loc[epoch,list_entropy_features] = ent_MSG + ent_MSD
            df_model_one_seizure.loc[epoch,'correlation'] = correlation
            
        df_model_one_seizure['evt_name'] = csv_name[:-4]

        df_model_all_seizure = pd.concat([df_model_all_seizure,df_model_one_seizure]) 
            
    else: 
        print(csv_name, ' is out of bornes')   
    
    if save_files:
        df_model_all_seizure.to_csv(DIR_SAVE_FEATURES + csv_saving_name)


i =  0
it =  1
i =  1
20100319T093900RIG.csv  is out of bornes
i =  2
20100319T104200RIG.csv  is out of bornes
i =  3
20100319T103600RIG.csv  is out of bornes
i =  4
20100319T094600RIG.csv  is out of bornes
i =  5
20100319T102954RIG.csv  is out of bornes
i =  6
it =  2
i =  7
it =  3
i =  8
it =  4
i =  9
it =  5
i =  10
it =  6
i =  11
it =  7
i =  12
it =  8
i =  13
it =  9
i =  14
it =  10
i =  15
it =  11
i =  16
it =  12
i =  17
it =  13
i =  18
it =  14
i =  19
it =  15
i =  20
it =  16
i =  21
it =  17
i =  22
it =  18
i =  23
20051130T105454MARMIR.csv  is out of bornes
i =  24
it =  19
i =  25
20100308T143600RHM.csv  is out of bornes
i =  26
20100308T153300RHM.csv  is out of bornes
i =  27
20100308T150400RHM.csv  is out of bornes
i =  28
20090825T094800TOT.csv  is out of bornes
i =  29
20090825T111900TOT.csv  is out of bornes
i =  30
20091120T095503KSY.csv  is out of bornes
i =  31
it =  20
i =  32
it =  21
i =  33
it =  22
i =  34
20100520T155400EXC.csv  is out of bornes
i =  

In [16]:
len(set(df_model_all_seizure.evt_name.tolist()))

5

In [18]:
member = 'MSD'
wavelet_type = 'db1'
wavelet_level = 5
'''
return a df for graf and e discret wavelet heatmap, arguments are : 
    - acc data csv name
    - member (MSG or MSD)
    - discret wavelet trype
    - number of coefficient level required
'''
df_graf = df_epoch.copy(deep = True) # pd.read_csv(csv_path,index_col = 0)

In [21]:
import pywt

In [23]:

df_graf.timeline = [round(i) - df_graf.timeline.iloc[0] for i in list(df_graf.timeline)]

np_x = df_graf[member].to_numpy()
np_x = np_x - np.mean(np_x) #pour ne pas charger les plus basses fréquences
    # np_x = np_x[10000:15000]
print(wavelet_type)
wv_signal = pywt.wavedec(np_x,wavelet_type,level = wavelet_level)
# df_frequency_for_heatmap = utils.return_frequency_dataframe(wv_signal)


db1


In [40]:
np.shape(wv_signal[0])

(8,)

In [12]:
df_toto = pd.DataFrame(columns=('a','b'))
df_toto.loc[0,'a'] = spectre_MSG
# df_toto.at[0,'a'] = spectre_MSG
df_toto


,a,b
0,"[0.0022614720597588876, 0.0011573748008248718,...",NaN


In [13]:
df_toto.loc[0,'a']

[0.0022614720597588876,
 0.0011573748008248718,
 0.001408700826141567,
 0.0022464764098959504,
 0.0018244359814252945,
 0.001211264730814184,
 0.0017406616091007634,
 0.001337676323693058,
 0.0009383201028620341,
 0.0011237615776631587,
 0.0011132080156511398,
 0.0011128249370068609]

In [64]:
aa[0]

0    [[0.0014297593020164343, 0.0016994897945905342...
0    [[0.0014297593020164343, 0.0016994897945905342...
Name: spectre_MSG, dtype: object

In [49]:
np.shape([[2,3]])

(1, 2)

In [31]:
np.shape(np.transpose(np.array([2,3])))

(2,)

In [ ]:
df_model_all_seizure

In [ ]:
def loc_get_epoch_df(df,no_epoch,duree_epoch):
    
    '''
    return a df of duree_epoch long centerd on 
    no_epoch * duree_epoch / 2
    '''
    window_overlap = duree_epoch / 2
    df_loc = df[(df.timeline > no_epoch * window_overlap) & 
    (df.timeline < (no_epoch+2) * window_overlap)]
    
    return(df_loc)


In [ ]:
toto = return_norm_L1
toto([-2,2])